# 85. Pivot LB（三角不等式下界）による公平なフィルタ比較

## 目的
- 実験84の公平性問題を解決: 全パイプラインが同一のITQ 128bitハッシュを使用
- 三角不等式に基づくLB(q,x) = max_i |d(q,pi) - d(x,pi)| をpivotフィルタとして導入
- LB方式の利点: 閾値チューニング不要、理論的保証あり、Hamming全探索の~16x低コスト

## 比較パイプライン (全てITQハッシュ固定)
| ID | Pipeline | Pre-filter | Description |
|----|----------|-----------|-------------|
| A  | Hamming全探索 | None | Baseline: Hamming top-L → Cosine rerank |
| B  | PivotRange(t=20) | per-pivot ±20 | 現行方式 |
| C  | PivotLB(top-L) | LB sort → top-L | **新方式**: LBで候補をL件に絞る |
| D  | Band(bw=8) | band match | Band転置インデックス |
| E  | Band+Probe(8) | confidence probe | 8 probes |
| F  | Band+Probe(16) | confidence probe | 16 probes |
| G  | Band+Probe(16)+PvtRange(25) | band+probe+range | 現行組合せ |
| H  | Band+Probe(16)+PvtLB | band+probe+LB | **新方式**: Band+LBカスケード |

## 理論的背景
三角不等式: d(q,x) >= |d(q,pi) - d(x,pi)| for any pivot pi  
∴ d(q,x) >= max_i |d(q,pi) - d(x,pi)| = LB(q,x)  

LBが小さい文書ほど真のNN候補として有望 → argpartitionでtop-L選択  
LB計算コスト: N×8演算 vs Hamming全探索 N×128演算 → ~16x安い

## 0. セットアップ

In [1]:
import sys
import numpy as np
import time
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')
from itq_lsh import ITQLSH, hamming_distance, hamming_distance_batch
from dflsh import build_band_index, band_filter, confidence_multiprobe

DATA_DIR = Path('../data')
np.random.seed(42)

N_QUERIES = 100
TOP_K = 10
CANDIDATE_LIMITS = [100, 500, 1000]
print(f'Configuration: N_QUERIES={N_QUERIES}, TOP_K={TOP_K}, CANDIDATE_LIMITS={CANDIDATE_LIMITS}')

Configuration: N_QUERIES=100, TOP_K=10, CANDIDATE_LIMITS=[100, 500, 1000]


## 1. データロード

In [2]:
datasets = {}

# English E5-base
datasets['EN'] = {
    'embeddings': np.load(DATA_DIR / '10k_e5_base_en_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_e5_base_en_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_e5_base_en_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_e5_base_en.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_e5_base_128bits.pkl')),
}

# Japanese E5-base
datasets['JA'] = {
    'embeddings': np.load(DATA_DIR / '10k_e5_base_ja_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_e5_base_ja_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_e5_base_ja_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_e5_base_ja.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_e5_base_128bits.pkl')),
}

# MiniLM
datasets['MiniLM'] = {
    'embeddings': np.load(DATA_DIR / '10k_minilm_embeddings.npy'),
    'hashes': np.load(DATA_DIR / '10k_minilm_hashes_128bits.npy'),
    'pivot_dist': np.load(DATA_DIR / '10k_minilm_pivot_distances.npy'),
    'pivots': np.load(DATA_DIR / 'pivots_8_minilm.npy'),
    'itq_model': ITQLSH.load(str(DATA_DIR / 'itq_minilm_128bits.pkl')),
}

for name, d in datasets.items():
    print(f'{name}: emb={d["embeddings"].shape}, hash={d["hashes"].shape}, '
          f'pivot_dist={d["pivot_dist"].shape}, pivots={d["pivots"].shape}')

EN: emb=(10000, 768), hash=(10000, 128), pivot_dist=(10000, 8), pivots=(8, 128)
JA: emb=(10000, 768), hash=(10000, 128), pivot_dist=(10000, 8), pivots=(8, 128)
MiniLM: emb=(10000, 384), hash=(10000, 128), pivot_dist=(10000, 8), pivots=(8, 128)


## 2. LB下界の品質分析

LB(q,x) = max_i |d(q,pi) - d(x,pi)| がHamming距離の良い近似になるか確認。  
LBとHamming距離のSpearman相関が高いほど、LBによるpre-sortが有効。

In [3]:
def analyze_lb_quality(hashes, pivot_distances, pivots, label, n_queries=100):
    """LB下界のHamming距離との相関を分析"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(hashes), n_queries, replace=False)
    
    all_lb = []
    all_hamming = []
    lb_tightness = []
    
    for qi in query_indices:
        query_pdists = np.array([
            hamming_distance(hashes[qi], p) for p in pivots
        ])
        
        # LB for all docs
        lb = np.max(np.abs(query_pdists[np.newaxis, :] - pivot_distances), axis=1)
        
        # True Hamming
        h_dists = hamming_distance_batch(hashes[qi], hashes).astype(float)
        
        # Exclude self
        mask = np.ones(len(hashes), dtype=bool)
        mask[qi] = False
        
        all_lb.extend(lb[mask])
        all_hamming.extend(h_dists[mask])
        
        # Tightness: LB/Hamming (closer to 1 = tighter bound)
        valid = h_dists[mask] > 0
        ratios = lb[mask][valid] / h_dists[mask][valid]
        lb_tightness.extend(ratios)
    
    corr, pval = spearmanr(all_lb, all_hamming)
    
    print(f'\n{label}:')
    print(f'  LB-Hamming Spearman: {corr:.4f} (p={pval:.2e})')
    print(f'  LB mean: {np.mean(all_lb):.1f}, Hamming mean: {np.mean(all_hamming):.1f}')
    print(f'  LB tightness (LB/Hamming): mean={np.mean(lb_tightness):.3f}, '
          f'median={np.median(lb_tightness):.3f}')
    print(f'  LB tightness percentiles: '
          f'10%={np.percentile(lb_tightness, 10):.3f}, '
          f'50%={np.percentile(lb_tightness, 50):.3f}, '
          f'90%={np.percentile(lb_tightness, 90):.3f}')
    frac_tight = np.mean(np.array(lb_tightness) >= 0.5)
    print(f'  Fraction where LB >= 50% of Hamming: {frac_tight:.1%}')
    
    return corr

print('='*70)
print('LB Quality Analysis: LB(q,x) = max_i |d(q,pi) - d(x,pi)| vs Hamming')
print('='*70)

lb_correlations = {}
for name, d in datasets.items():
    lb_correlations[name] = analyze_lb_quality(
        d['hashes'], d['pivot_dist'], d['pivots'], name
    )

LB Quality Analysis: LB(q,x) = max_i |d(q,pi) - d(x,pi)| vs Hamming



EN:
  LB-Hamming Spearman: 0.2613 (p=0.00e+00)
  LB mean: 14.5, Hamming mean: 52.4
  LB tightness (LB/Hamming): mean=0.278, median=0.269
  LB tightness percentiles: 10%=0.179, 50%=0.269, 90%=0.388
  Fraction where LB >= 50% of Hamming: 1.3%



JA:
  LB-Hamming Spearman: 0.3070 (p=0.00e+00)
  LB mean: 18.2, Hamming mean: 64.0
  LB tightness (LB/Hamming): mean=0.285, median=0.270
  LB tightness percentiles: 10%=0.173, 50%=0.270, 90%=0.412
  Fraction where LB >= 50% of Hamming: 3.6%



MiniLM:
  LB-Hamming Spearman: 0.2119 (p=0.00e+00)
  LB mean: 16.7, Hamming mean: 64.0
  LB tightness (LB/Hamming): mean=0.261, median=0.246
  LB tightness percentiles: 10%=0.159, 50%=0.246, 90%=0.365
  Fraction where LB >= 50% of Hamming: 1.8%


## 3. LB vs Range: 候補選択品質の直接比較

同じ候補数Lで、LBソート上位L件 vs Range(threshold)フィルタのFilter Recall / R@10を比較。  
LBアプローチの利点: 閾値不要で候補数を直接制御可能。

In [4]:
def get_ground_truth(embeddings, qi, top_k=10):
    cos_sims = cosine_similarity(embeddings[qi:qi+1], embeddings)[0]
    cos_sims[qi] = -1
    return set(np.argsort(cos_sims)[-top_k:])


def compare_lb_vs_range(embeddings, hashes, pivot_distances, pivots, label,
                        candidate_limits=[100, 500, 1000]):
    """LB top-L vs Range filter のFilter Recall / R@10直接比較"""
    rng = np.random.default_rng(42)
    query_indices = rng.choice(len(embeddings), N_QUERIES, replace=False)
    
    results = []
    
    for L in candidate_limits:
        lb_filter_recalls = []
        lb_final_recalls = []
        range_filter_recalls = {t: [] for t in [15, 20, 25, 30]}
        range_final_recalls = {t: [] for t in [15, 20, 25, 30]}
        range_cand_counts = {t: [] for t in [15, 20, 25, 30]}
        
        for qi in query_indices:
            gt = get_ground_truth(embeddings, qi, TOP_K)
            query_hash = hashes[qi]
            query_pdists = np.array([hamming_distance(query_hash, p) for p in pivots])
            
            # --- LB approach ---
            lb = np.max(np.abs(query_pdists[np.newaxis, :] - pivot_distances), axis=1)
            lb[qi] = 999
            
            if L < len(lb):
                lb_top_indices = np.argpartition(lb, L)[:L]
            else:
                lb_top_indices = np.arange(len(lb))
                lb_top_indices = lb_top_indices[lb_top_indices != qi]
            
            lb_filter_recalls.append(len(gt & set(lb_top_indices)) / TOP_K)
            
            # Hamming sort within LB candidates -> cosine rerank
            h_dists = hamming_distance_batch(query_hash, hashes[lb_top_indices])
            ham_top = np.argsort(h_dists)[:L]
            final_cands = lb_top_indices[ham_top]
            cand_cos = cosine_similarity(embeddings[qi:qi+1], embeddings[final_cands])[0]
            top_rerank = final_cands[np.argsort(cand_cos)[-TOP_K:]]
            lb_final_recalls.append(len(gt & set(top_rerank)) / TOP_K)
            
            # --- Range approach at various thresholds ---
            for t in [15, 20, 25, 30]:
                mask = np.ones(len(hashes), dtype=bool)
                mask[qi] = False
                for i in range(len(pivots)):
                    lower = query_pdists[i] - t
                    upper = query_pdists[i] + t
                    mask &= (pivot_distances[:, i] >= lower) & (pivot_distances[:, i] <= upper)
                
                range_cands = np.where(mask)[0]
                range_cand_counts[t].append(len(range_cands))
                range_filter_recalls[t].append(len(gt & set(range_cands)) / TOP_K)
                
                if len(range_cands) > 0:
                    h_d = hamming_distance_batch(query_hash, hashes[range_cands])
                    top_idx = np.argsort(h_d)[:L]
                    fc = range_cands[top_idx]
                    cc = cosine_similarity(embeddings[qi:qi+1], embeddings[fc])[0]
                    tr = fc[np.argsort(cc)[-TOP_K:]]
                    range_final_recalls[t].append(len(gt & set(tr)) / TOP_K)
                else:
                    range_final_recalls[t].append(0.0)
        
        results.append({
            'L': L,
            'lb_filter_recall': np.mean(lb_filter_recalls),
            'lb_final_recall': np.mean(lb_final_recalls),
            'range': {t: {
                'cands': np.mean(range_cand_counts[t]),
                'filter_recall': np.mean(range_filter_recalls[t]),
                'final_recall': np.mean(range_final_recalls[t]),
            } for t in [15, 20, 25, 30]}
        })
    
    print(f'\n--- {label} ---')
    for r in results:
        L = r['L']
        print(f'\n  candidate_limit L={L}:')
        print(f'    {"Method":<25} {"Cands":>8} {"FiltRcl":>8} {"R@10":>8}')
        print(f'    {"-"*50}')
        print(f'    {"PivotLB(top-L)":<25} {L:>8} '
              f'{r["lb_filter_recall"]*100:>7.1f}% {r["lb_final_recall"]*100:>7.1f}%')
        for t in [15, 20, 25, 30]:
            rr = r['range'][t]
            print(f'    {f"PivotRange(t={t})":<25} {rr["cands"]:>7.0f} '
                  f'{rr["filter_recall"]*100:>7.1f}% {rr["final_recall"]*100:>7.1f}%')
    
    return results

print('='*70)
print('LB vs Range Direct Comparison')
print('='*70)

lb_vs_range_results = {}
for name, d in datasets.items():
    lb_vs_range_results[name] = compare_lb_vs_range(
        d['embeddings'], d['hashes'], d['pivot_dist'], d['pivots'], name
    )

LB vs Range Direct Comparison



--- EN ---

  candidate_limit L=100:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 100     7.9%     7.9%
    PivotRange(t=15)             6317    90.1%    59.0%
    PivotRange(t=20)             9055    99.2%    61.4%
    PivotRange(t=25)             9825    99.9%    60.6%
    PivotRange(t=30)             9969    99.9%    61.7%

  candidate_limit L=500:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 500    23.0%    23.0%
    PivotRange(t=15)             6317    90.1%    78.1%
    PivotRange(t=20)             9055    99.2%    84.2%
    PivotRange(t=25)             9825    99.9%    84.0%
    PivotRange(t=30)             9969    99.9%    84.1%

  candidate_limit L=1000:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB


--- JA ---

  candidate_limit L=100:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 100    16.2%    16.2%
    PivotRange(t=15)             3842    87.4%    76.6%
    PivotRange(t=20)             6956    98.1%    81.8%
    PivotRange(t=25)             8738    99.9%    83.1%
    PivotRange(t=30)             9486    99.9%    83.1%

  candidate_limit L=500:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 500    40.8%    40.8%
    PivotRange(t=15)             3842    87.4%    86.9%
    PivotRange(t=20)             6956    98.1%    96.7%
    PivotRange(t=25)             8738    99.9%    98.0%
    PivotRange(t=30)             9486    99.9%    97.9%

  candidate_limit L=1000:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB


--- MiniLM ---

  candidate_limit L=100:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 100     9.3%     9.3%
    PivotRange(t=15)             4903    84.5%    77.1%
    PivotRange(t=20)             8025    96.7%    85.7%
    PivotRange(t=25)             9360    98.8%    87.1%
    PivotRange(t=30)             9767    98.9%    86.9%

  candidate_limit L=500:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    PivotLB(top-L)                 500    28.0%    28.0%
    PivotRange(t=15)             4903    84.5%    83.5%
    PivotRange(t=20)             8025    96.7%    95.2%
    PivotRange(t=25)             9360    98.8%    97.1%
    PivotRange(t=30)             9767    98.9%    97.2%

  candidate_limit L=1000:
    Method                       Cands  FiltRcl     R@10
    --------------------------------------------------
    Piv

## 4. 統一評価関数

全8パイプラインを公平に評価する統一関数。  
全パイプラインがITQハッシュを共有し、フィルタメカニズムのみが異なる。  
新指標: Hamming計算量（実際に計算したHamming距離の数）を追跡。

In [5]:
def evaluate_all_pipelines(data, dataset_name, candidate_limit=500):
    """全8パイプラインを統一評価"""
    emb = data['embeddings']
    hashes = data['hashes']
    pivot_dist = data['pivot_dist']
    pivots = data['pivots']
    itq_model = data['itq_model']
    N = len(emb)
    
    rng = np.random.default_rng(42)
    query_indices = rng.choice(N, N_QUERIES, replace=False)
    
    # Pre-compute band index and projections
    bi = build_band_index(hashes, 8)
    _, all_projections = itq_model.transform_with_confidence(emb)
    
    # Pre-compute ground truth
    all_gt = {}
    for qi in query_indices:
        all_gt[qi] = get_ground_truth(emb, qi, TOP_K)
    
    pipeline_defs = [
        ('A', 'Hamming全探索'),
        ('B', 'PivotRange(t=20)'),
        ('C', 'PivotLB(top-L)'),
        ('D', 'Band(bw=8)'),
        ('E', 'Band+Probe(8)'),
        ('F', 'Band+Probe(16)'),
        ('G', 'Band+Probe(16)+PvtRange(25)'),
        ('H', 'Band+Probe(16)+PvtLB'),
    ]
    
    all_results = []
    
    for pid, pname in pipeline_defs:
        filter_recalls = []
        final_recalls = []
        filter_counts = []
        hamming_computed_counts = []
        total_times = []
        
        for qi in query_indices:
            gt = all_gt[qi]
            query_hash = hashes[qi]
            
            start = time.time()
            
            query_pdists = np.array([
                hamming_distance(query_hash, p) for p in pivots
            ])
            
            # ============ STAGE 1: Candidate selection ============
            if pid == 'A':
                cands = np.arange(N)
                cands = cands[cands != qi]
                
            elif pid == 'B':
                mask = np.ones(N, dtype=bool)
                mask[qi] = False
                for i in range(len(pivots)):
                    lower = query_pdists[i] - 20
                    upper = query_pdists[i] + 20
                    mask &= (pivot_dist[:, i] >= lower) & (pivot_dist[:, i] <= upper)
                cands = np.where(mask)[0]
                
            elif pid == 'C':
                lb = np.max(np.abs(query_pdists[np.newaxis, :] - pivot_dist), axis=1)
                lb[qi] = 999
                if candidate_limit < N:
                    top_lb = np.argpartition(lb, candidate_limit)[:candidate_limit]
                else:
                    top_lb = np.arange(N)
                    top_lb = top_lb[top_lb != qi]
                cands = top_lb
                
            elif pid == 'D':
                cands = band_filter(query_hash, bi, 8, min_matches=1)
                cands = cands[cands != qi]
                
            elif pid == 'E':
                query_proj = all_projections[qi]
                cands = confidence_multiprobe(
                    query_hash, query_proj, bi, 8, max_probes=8, order='confidence'
                )
                cands = cands[cands != qi]
                
            elif pid == 'F':
                query_proj = all_projections[qi]
                cands = confidence_multiprobe(
                    query_hash, query_proj, bi, 8, max_probes=16, order='confidence'
                )
                cands = cands[cands != qi]
                
            elif pid == 'G':
                query_proj = all_projections[qi]
                cands = confidence_multiprobe(
                    query_hash, query_proj, bi, 8, max_probes=16, order='confidence'
                )
                cands = cands[cands != qi]
                if len(cands) > 0:
                    cand_pdists = pivot_dist[cands]
                    mask = np.ones(len(cands), dtype=bool)
                    for i in range(len(pivots)):
                        lower = query_pdists[i] - 25
                        upper = query_pdists[i] + 25
                        mask &= (cand_pdists[:, i] >= lower) & (cand_pdists[:, i] <= upper)
                    cands = cands[mask]
                    
            elif pid == 'H':
                query_proj = all_projections[qi]
                cands = confidence_multiprobe(
                    query_hash, query_proj, bi, 8, max_probes=16, order='confidence'
                )
                cands = cands[cands != qi]
                if len(cands) > candidate_limit:
                    cand_lb = np.max(
                        np.abs(query_pdists[np.newaxis, :] - pivot_dist[cands]), axis=1
                    )
                    lb_top = np.argpartition(cand_lb, candidate_limit)[:candidate_limit]
                    cands = cands[lb_top]
            
            filter_counts.append(len(cands))
            filter_recalls.append(len(gt & set(cands)) / TOP_K)
            
            # ============ STAGE 2: Hamming sort + Cosine rerank ============
            if len(cands) > 0:
                h_dists = hamming_distance_batch(query_hash, hashes[cands])
                hamming_computed_counts.append(len(cands))
                top_idx = np.argsort(h_dists)[:candidate_limit]
                final_cands = cands[top_idx]
                
                cand_cos = cosine_similarity(emb[qi:qi+1], emb[final_cands])[0]
                top_in_cand = final_cands[np.argsort(cand_cos)[-TOP_K:]]
                final_recalls.append(len(gt & set(top_in_cand)) / TOP_K)
            else:
                hamming_computed_counts.append(0)
                final_recalls.append(0.0)
            
            total_times.append(time.time() - start)
        
        all_results.append({
            'id': pid,
            'name': pname,
            'filter_candidates': np.mean(filter_counts),
            'filter_candidates_std': np.std(filter_counts),
            'reduction': 1 - np.mean(filter_counts) / N,
            'filter_recall': np.mean(filter_recalls),
            'recall_at_k': np.mean(final_recalls),
            'hamming_computed': np.mean(hamming_computed_counts),
            'time_ms': np.mean(total_times) * 1000,
        })
    
    return all_results

## 5. 全パイプライン比較 (candidate_limit sweep)

In [6]:
def print_results_table(results, dataset_name, candidate_limit):
    """結果テーブルを表示"""
    print(f'\n{dataset_name} (L={candidate_limit})')
    print(f'{"ID":<3} {"Pipeline":<28} {"Cands":>7} {"Reduct":>7} '
          f'{"FiltRcl":>8} {"R@10":>7} {"HamCalc":>8} {"Time":>8}')
    print('-' * 85)
    for r in results:
        print(f'{r["id"]:<3} {r["name"]:<28} {r["filter_candidates"]:>6.0f} '
              f'{r["reduction"]*100:>6.1f}% '
              f'{r["filter_recall"]*100:>7.1f}% '
              f'{r["recall_at_k"]*100:>6.1f}% '
              f'{r["hamming_computed"]:>7.0f} '
              f'{r["time_ms"]:>7.2f}')


print('='*85)
print('Fair Pipeline Comparison (All ITQ 128bit, Filter Mechanism Only)')
print('='*85)

all_results = {}

for dataset_name in ['EN', 'JA', 'MiniLM']:
    print(f'\n{"="*40}')
    print(f'  Dataset: {dataset_name}')
    print(f'{"="*40}')
    
    for L in CANDIDATE_LIMITS:
        results = evaluate_all_pipelines(datasets[dataset_name], dataset_name,
                                          candidate_limit=L)
        print_results_table(results, dataset_name, L)
        all_results[f'{dataset_name}_L{L}'] = results

Fair Pipeline Comparison (All ITQ 128bit, Filter Mechanism Only)

  Dataset: EN



EN (L=100)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   61.0%    9999    1.92
B   PivotRange(t=20)               9055    9.4%    99.2%   61.4%    9055    1.73
C   PivotLB(top-L)                  100   99.0%     7.9%    7.9%     100    0.84
D   Band(bw=8)                     2181   78.2%    68.9%   53.1%    2181    1.06
E   Band+Probe(8)                  2793   72.1%    75.6%   56.4%    2793    1.33
F   Band+Probe(16)                 3757   62.4%    85.1%   59.7%    3757    1.64
G   Band+Probe(16)+PvtRange(25)    3713   62.9%    85.0%   59.6%    3713    1.71
H   Band+Probe(16)+PvtLB            100   99.0%    11.7%   11.6%     100    1.24



EN (L=500)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   84.0%    9999    2.40
B   PivotRange(t=20)               9055    9.4%    99.2%   84.2%    9055    2.23
C   PivotLB(top-L)                  500   95.0%    23.0%   23.0%     500    1.46
D   Band(bw=8)                     2181   78.2%    68.9%   66.0%    2181    1.65
E   Band+Probe(8)                  2793   72.1%    75.6%   71.4%    2793    1.96
F   Band+Probe(16)                 3757   62.4%    85.1%   78.0%    3757    2.25
G   Band+Probe(16)+PvtRange(25)    3713   62.9%    85.0%   77.8%    3713    2.33
H   Band+Probe(16)+PvtLB            500   95.0%    33.4%   33.3%     500    2.00



EN (L=1000)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   91.3%    9999    3.86
B   PivotRange(t=20)               9055    9.4%    99.2%   91.1%    9055    3.87
C   PivotLB(top-L)                 1000   90.0%    34.7%   34.6%    1000    2.78
D   Band(bw=8)                     2181   78.2%    68.9%   68.6%    2181    3.01
E   Band+Probe(8)                  2793   72.1%    75.6%   75.0%    2793    3.42
F   Band+Probe(16)                 3757   62.4%    85.1%   82.7%    3757    4.29
G   Band+Probe(16)+PvtRange(25)    3713   62.9%    85.0%   82.5%    3713    3.97
H   Band+Probe(16)+PvtLB           1000   90.0%    51.2%   51.1%    1000    3.56

  Dataset: JA



JA (L=100)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   83.5%    9999    1.95
B   PivotRange(t=20)               6956   30.4%    98.1%   81.8%    6956    1.51
C   PivotLB(top-L)                  100   99.0%    16.2%   16.2%     100    0.84
D   Band(bw=8)                      694   93.1%    63.9%   61.9%     694    0.75
E   Band+Probe(8)                  1002   90.0%    72.4%   69.0%    1002    0.98
F   Band+Probe(16)                 1318   86.8%    82.9%   77.4%    1318    1.10
G   Band+Probe(16)+PvtRange(25)    1221   87.8%    82.8%   76.9%    1221    1.14
H   Band+Probe(16)+PvtLB            100   99.0%    29.6%   29.6%     100    0.97



JA (L=500)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   98.1%    9999    2.41
B   PivotRange(t=20)               6956   30.4%    98.1%   96.7%    6956    2.00
C   PivotLB(top-L)                  500   95.0%    40.8%   40.8%     500    1.45
D   Band(bw=8)                      694   93.1%    63.9%   63.9%     694    1.29
E   Band+Probe(8)                  1002   90.0%    72.4%   72.4%    1002    1.54
F   Band+Probe(16)                 1318   86.8%    82.9%   82.7%    1318    1.66
G   Band+Probe(16)+PvtRange(25)    1221   87.8%    82.8%   82.6%    1221    1.71
H   Band+Probe(16)+PvtLB            500   95.0%    67.0%   67.0%     500    1.61



JA (L=1000)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   99.0%    9999    3.88
B   PivotRange(t=20)               6956   30.4%    98.1%   97.6%    6956    3.51
C   PivotLB(top-L)                 1000   90.0%    55.6%   55.6%    1000    2.79
D   Band(bw=8)                      694   93.1%    63.9%   63.9%     694    1.71
E   Band+Probe(8)                  1002   90.0%    72.4%   72.4%    1002    2.61
F   Band+Probe(16)                 1318   86.8%    82.9%   82.9%    1318    3.21
G   Band+Probe(16)+PvtRange(25)    1221   87.8%    82.8%   82.8%    1221    3.29
H   Band+Probe(16)+PvtLB           1000   90.0%    81.8%   81.8%    1000    3.04

  Dataset: MiniLM



MiniLM (L=100)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   87.7%    9999    1.94
B   PivotRange(t=20)               8025   19.8%    96.7%   85.7%    8025    1.62
C   PivotLB(top-L)                  100   99.0%     9.3%    9.3%     100    0.85
D   Band(bw=8)                      654   93.5%    50.9%   50.5%     654    0.71
E   Band+Probe(8)                   955   90.5%    59.5%   58.3%     955    0.94
F   Band+Probe(16)                 1250   87.5%    71.4%   68.9%    1250    1.04
G   Band+Probe(16)+PvtRange(25)    1192   88.1%    70.3%   67.8%    1192    1.10
H   Band+Probe(16)+PvtLB            100   99.0%    22.6%   22.6%     100    0.94



MiniLM (L=500)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   98.3%    9999    2.35
B   PivotRange(t=20)               8025   19.8%    96.7%   95.2%    8025    2.07
C   PivotLB(top-L)                  500   95.0%    28.0%   28.0%     500    1.42
D   Band(bw=8)                      654   93.5%    50.9%   50.9%     654    1.24
E   Band+Probe(8)                   955   90.5%    59.5%   59.5%     955    1.47
F   Band+Probe(16)                 1250   87.5%    71.4%   71.4%    1250    1.59
G   Band+Probe(16)+PvtRange(25)    1192   88.1%    70.3%   70.3%    1192    1.64
H   Band+Probe(16)+PvtLB            500   95.0%    53.5%   53.5%     500    1.55



MiniLM (L=1000)
ID  Pipeline                       Cands  Reduct  FiltRcl    R@10  HamCalc     Time
-------------------------------------------------------------------------------------
A   Hamming全探索                     9999    0.0%   100.0%   99.5%    9999    3.18
B   PivotRange(t=20)               8025   19.8%    96.7%   96.3%    8025    2.86
C   PivotLB(top-L)                 1000   90.0%    39.7%   39.7%    1000    2.20
D   Band(bw=8)                      654   93.5%    50.9%   50.9%     654    1.23
E   Band+Probe(8)                   955   90.5%    59.5%   59.5%     955    1.87
F   Band+Probe(16)                 1250   87.5%    71.4%   71.4%    1250    2.33
G   Band+Probe(16)+PvtRange(25)    1192   88.1%    70.3%   70.3%    1192    2.38
H   Band+Probe(16)+PvtLB           1000   90.0%    68.1%   68.1%    1000    2.36


## 6. PivotLB vs PivotRange 直接比較分析

核心の問い: LB方式はRange方式より良いか?
- 同じ候補数での Filter Recall / R@10
- LBは閾値不要 (threshold tuning不要)
- LBは候補数を正確に制御可能

In [7]:
print('='*85)
print('PivotLB vs PivotRange: Head-to-Head Comparison')
print('='*85)

for dataset_name in ['EN', 'JA', 'MiniLM']:
    print(f'\n--- {dataset_name} ---')
    print(f'{"L":>6} | {"--- PivotRange(t=20) [B] ---":^32} | {"--- PivotLB(top-L) [C] ---":^32} | {"Delta":>6}')
    print(f'{"":>6} | {"Cands":>8} {"FiltRcl":>8} {"R@10":>8} | '
          f'{"Cands":>8} {"FiltRcl":>8} {"R@10":>8} | {"dR@10":>6}')
    print('-' * 90)
    
    for L in CANDIDATE_LIMITS:
        results = all_results[f'{dataset_name}_L{L}']
        r_range = next(r for r in results if r['id'] == 'B')
        r_lb = next(r for r in results if r['id'] == 'C')
        delta = (r_lb['recall_at_k'] - r_range['recall_at_k']) * 100
        
        print(f'{L:>6} | '
              f'{r_range["filter_candidates"]:>7.0f} '
              f'{r_range["filter_recall"]*100:>7.1f}% '
              f'{r_range["recall_at_k"]*100:>7.1f}% | '
              f'{r_lb["filter_candidates"]:>7.0f} '
              f'{r_lb["filter_recall"]*100:>7.1f}% '
              f'{r_lb["recall_at_k"]*100:>7.1f}% | '
              f'{delta:>+5.1f}%')

print(f'\n{"="*90}')
print('Band+Probe(16)+PivotLB [H] vs Band+Probe(16)+PivotRange(25) [G]')
print('='*90)

for dataset_name in ['EN', 'JA', 'MiniLM']:
    print(f'\n--- {dataset_name} ---')
    print(f'{"L":>6} | {"--- +PvtRange(25) [G] ---":^32} | {"--- +PvtLB [H] ---":^32} | {"Delta":>6}')
    print(f'{"":>6} | {"Cands":>8} {"FiltRcl":>8} {"R@10":>8} | '
          f'{"Cands":>8} {"FiltRcl":>8} {"R@10":>8} | {"dR@10":>6}')
    print('-' * 90)
    
    for L in CANDIDATE_LIMITS:
        results = all_results[f'{dataset_name}_L{L}']
        r_g = next(r for r in results if r['id'] == 'G')
        r_h = next(r for r in results if r['id'] == 'H')
        delta = (r_h['recall_at_k'] - r_g['recall_at_k']) * 100
        
        print(f'{L:>6} | '
              f'{r_g["filter_candidates"]:>7.0f} '
              f'{r_g["filter_recall"]*100:>7.1f}% '
              f'{r_g["recall_at_k"]*100:>7.1f}% | '
              f'{r_h["filter_candidates"]:>7.0f} '
              f'{r_h["filter_recall"]*100:>7.1f}% '
              f'{r_h["recall_at_k"]*100:>7.1f}% | '
              f'{delta:>+5.1f}%')

PivotLB vs PivotRange: Head-to-Head Comparison

--- EN ---
     L |   --- PivotRange(t=20) [B] ---   |    --- PivotLB(top-L) [C] ---    |  Delta
       |    Cands  FiltRcl     R@10 |    Cands  FiltRcl     R@10 |  dR@10
------------------------------------------------------------------------------------------
   100 |    9055    99.2%    61.4% |     100     7.9%     7.9% | -53.5%
   500 |    9055    99.2%    84.2% |     500    23.0%    23.0% | -61.2%
  1000 |    9055    99.2%    91.1% |    1000    34.7%    34.6% | -56.5%

--- JA ---
     L |   --- PivotRange(t=20) [B] ---   |    --- PivotLB(top-L) [C] ---    |  Delta
       |    Cands  FiltRcl     R@10 |    Cands  FiltRcl     R@10 |  dR@10
------------------------------------------------------------------------------------------
   100 |    6956    98.1%    81.8% |     100    16.2%    16.2% | -65.6%
   500 |    6956    98.1%    96.7% |     500    40.8%    40.8% | -55.9%
  1000 |    6956    98.1%    97.6% |    1000    55.6%    55.6% | -4

## 7. Pareto最適フロンティア分析

2軸: Recall@10 (高い方が良い) vs Hamming計算量 (低い方が良い)  
LBパイプラインがPareto上に乗るか?

In [8]:
def find_pareto(results):
    """Pareto最適解を特定（Recall最大化 & Hamming計算量最小化）"""
    pareto = []
    for r in results:
        dominated = False
        for other in results:
            if other is r:
                continue
            if (other['recall_at_k'] >= r['recall_at_k'] and 
                other['hamming_computed'] <= r['hamming_computed'] and
                (other['recall_at_k'] > r['recall_at_k'] or 
                 other['hamming_computed'] < r['hamming_computed'])):
                dominated = True
                break
        if not dominated:
            pareto.append(r)
    return sorted(pareto, key=lambda x: x['recall_at_k'], reverse=True)


print('='*85)
print('Pareto Optimal Frontier (Recall@10 vs Hamming Computation)')
print('='*85)

for L in [500]:
    for dataset_name in ['EN', 'JA', 'MiniLM']:
        results = all_results[f'{dataset_name}_L{L}']
        pareto = find_pareto(results)
        
        print(f'\n--- {dataset_name} (L={L}) ---')
        print(f'Pareto optimal ({len(pareto)}/{len(results)}):')
        for r in pareto:
            marker = ' ** NEW' if r['id'] in ('C', 'H') else ''
            print(f'  {r["id"]}: {r["name"]:<28} R@10={r["recall_at_k"]*100:.1f}%, '
                  f'HamCalc={r["hamming_computed"]:.0f}{marker}')
        
        non_pareto = [r for r in results if r not in pareto]
        if non_pareto:
            print(f'Non-Pareto:')
            for r in non_pareto:
                print(f'  {r["id"]}: {r["name"]:<28} R@10={r["recall_at_k"]*100:.1f}%, '
                      f'HamCalc={r["hamming_computed"]:.0f}')

Pareto Optimal Frontier (Recall@10 vs Hamming Computation)

--- EN (L=500) ---
Pareto optimal (6/8):
  B: PivotRange(t=20)             R@10=84.2%, HamCalc=9055
  F: Band+Probe(16)               R@10=78.0%, HamCalc=3757
  G: Band+Probe(16)+PvtRange(25)  R@10=77.8%, HamCalc=3713
  E: Band+Probe(8)                R@10=71.4%, HamCalc=2793
  D: Band(bw=8)                   R@10=66.0%, HamCalc=2181
  H: Band+Probe(16)+PvtLB         R@10=33.3%, HamCalc=500 ** NEW
Non-Pareto:
  A: Hamming全探索                   R@10=84.0%, HamCalc=9999
  C: PivotLB(top-L)               R@10=23.0%, HamCalc=500

--- JA (L=500) ---
Pareto optimal (6/8):
  A: Hamming全探索                   R@10=98.1%, HamCalc=9999
  B: PivotRange(t=20)             R@10=96.7%, HamCalc=6956
  F: Band+Probe(16)               R@10=82.7%, HamCalc=1318
  G: Band+Probe(16)+PvtRange(25)  R@10=82.6%, HamCalc=1221
  E: Band+Probe(8)                R@10=72.4%, HamCalc=1002
  H: Band+Probe(16)+PvtLB         R@10=67.0%, HamCalc=500 ** NEW
Non-Pare

## 8. 計算コスト分析

LBの計算コスト優位性を定量評価:
- LB計算: N × 8 uint8演算 (abs + max)
- Hamming全探索: N × 128 uint8比較
- LB/Hamming コスト比: ~1/16

In [9]:
print('='*85)
print('Computation Cost Analysis')
print('='*85)

N = 10000
P = 8
B = 128

print(f'\nParameters: N={N}, P={P} pivots, B={B} bits')
print(f'\nOperations per query:')
print(f'  LB computation:     N * P = {N} * {P} = {N*P:>10,} uint8 ops')
print(f'  Hamming full scan:  N * B = {N} * {B} = {N*B:>10,} uint8 ops')
print(f'  LB / Hamming ratio: {P/B:.4f} ({B//P}x cheaper)')

print(f'\nPipeline C (PivotLB top-L) total cost:')
for L in CANDIDATE_LIMITS:
    lb_cost = N * P
    hamming_cost = L * B
    total = lb_cost + hamming_cost
    baseline_cost = N * B
    savings = (1 - total / baseline_cost) * 100
    print(f'  L={L:>5}: LB={lb_cost:>8,} + Hamming(L)={hamming_cost:>8,} = '
          f'{total:>10,} ops ({savings:>5.1f}% savings vs baseline {baseline_cost:>10,})')

print(f'\nPipeline H (Band+Probe+PivotLB) total cost:')
for L in CANDIDATE_LIMITS:
    for dataset_name in ['EN', 'JA', 'MiniLM']:
        results = all_results[f'{dataset_name}_L{L}']
        r_h = next(r for r in results if r['id'] == 'H')
        r_f = next(r for r in results if r['id'] == 'F')
        band_cands = r_f['filter_candidates']
        lb_cost = band_cands * P
        hamming_cost = r_h['hamming_computed'] * B
        total = lb_cost + hamming_cost
        baseline_cost = N * B
        savings = (1 - total / baseline_cost) * 100
        print(f'  {dataset_name} L={L}: band_cands={band_cands:.0f} -> '
              f'LB={lb_cost:.0f} + Ham={hamming_cost:.0f} = {total:.0f} '
              f'({savings:.1f}% savings)')

Computation Cost Analysis

Parameters: N=10000, P=8 pivots, B=128 bits

Operations per query:
  LB computation:     N * P = 10000 * 8 =     80,000 uint8 ops
  Hamming full scan:  N * B = 10000 * 128 =  1,280,000 uint8 ops
  LB / Hamming ratio: 0.0625 (16x cheaper)

Pipeline C (PivotLB top-L) total cost:
  L=  100: LB=  80,000 + Hamming(L)=  12,800 =     92,800 ops ( 92.8% savings vs baseline  1,280,000)
  L=  500: LB=  80,000 + Hamming(L)=  64,000 =    144,000 ops ( 88.8% savings vs baseline  1,280,000)
  L= 1000: LB=  80,000 + Hamming(L)= 128,000 =    208,000 ops ( 83.8% savings vs baseline  1,280,000)

Pipeline H (Band+Probe+PivotLB) total cost:
  EN L=100: band_cands=3757 -> LB=30055 + Ham=12800 = 42855 (96.7% savings)
  JA L=100: band_cands=1318 -> LB=10548 + Ham=12800 = 23348 (98.2% savings)
  MiniLM L=100: band_cands=1250 -> LB=9997 + Ham=12800 = 22797 (98.2% savings)
  EN L=500: band_cands=3757 -> LB=30055 + Ham=64000 = 94055 (92.7% savings)
  JA L=500: band_cands=1318 -> LB=105

## 9. データセット横断比較

In [10]:
print('='*85)
print('Cross-Dataset Comparison at L=500')
print('='*85)

print(f'\n{"Dataset":<10} | {"Pipeline":<28} | {"R@10":>7} {"FiltRcl":>8} {"Cands":>7} {"HamCalc":>8}')
print('-' * 80)

for dataset_name in ['EN', 'JA', 'MiniLM']:
    results = all_results[f'{dataset_name}_L500']
    for r in results:
        print(f'{dataset_name:<10} | {r["name"]:<28} | '
              f'{r["recall_at_k"]*100:>6.1f}% {r["filter_recall"]*100:>7.1f}% '
              f'{r["filter_candidates"]:>6.0f} {r["hamming_computed"]:>7.0f}')
    print()

# Summary: best pipeline per dataset
print(f'\n{"="*85}')
print('Best Pipeline by Dataset (L=500)')
print('='*85)

for dataset_name in ['EN', 'JA', 'MiniLM']:
    results = all_results[f'{dataset_name}_L500']
    baseline = next(r for r in results if r['id'] == 'A')
    
    best = max(results, key=lambda x: x['recall_at_k'])
    
    efficient = [r for r in results if r['reduction'] > 0.5]
    best_efficient = max(efficient, key=lambda x: x['recall_at_k']) if efficient else None
    
    very_efficient = [r for r in results if r['reduction'] > 0.9]
    best_very = max(very_efficient, key=lambda x: x['recall_at_k']) if very_efficient else None
    
    print(f'\n[{dataset_name}]')
    print(f'  Baseline (A):      R@10={baseline["recall_at_k"]*100:.1f}%')
    print(f'  Best overall:      {best["id"]}: {best["name"]:<28} R@10={best["recall_at_k"]*100:.1f}%')
    if best_efficient:
        print(f'  Best >50% reduc:   {best_efficient["id"]}: {best_efficient["name"]:<28} '
              f'R@10={best_efficient["recall_at_k"]*100:.1f}%, Red={best_efficient["reduction"]*100:.1f}%')
    if best_very:
        print(f'  Best >90% reduc:   {best_very["id"]}: {best_very["name"]:<28} '
              f'R@10={best_very["recall_at_k"]*100:.1f}%, Red={best_very["reduction"]*100:.1f}%')

Cross-Dataset Comparison at L=500

Dataset    | Pipeline                     |    R@10  FiltRcl   Cands  HamCalc
--------------------------------------------------------------------------------
EN         | Hamming全探索                   |   84.0%   100.0%   9999    9999
EN         | PivotRange(t=20)             |   84.2%    99.2%   9055    9055
EN         | PivotLB(top-L)               |   23.0%    23.0%    500     500
EN         | Band(bw=8)                   |   66.0%    68.9%   2181    2181
EN         | Band+Probe(8)                |   71.4%    75.6%   2793    2793
EN         | Band+Probe(16)               |   78.0%    85.1%   3757    3757
EN         | Band+Probe(16)+PvtRange(25)  |   77.8%    85.0%   3713    3713
EN         | Band+Probe(16)+PvtLB         |   33.3%    33.4%    500     500

JA         | Hamming全探索                   |   98.1%   100.0%   9999    9999
JA         | PivotRange(t=20)             |   96.7%    98.1%   6956    6956
JA         | PivotLB(top-L)               |  

## 10. 結論とまとめ

In [11]:
print('='*85)
print('Experiment 85: Pivot LB Fair Comparison - Summary')
print('='*85)

print('\n【実験84からの改善点】')
print('1. 公平性: 全パイプラインがITQ 128bitハッシュを共有（ハッシュ品質差を排除）')
print('2. 新手法: LB(q,x) = max_i |d(q,pi)-d(x,pi)| による三角不等式下界フィルタ')
print('3. 評価軸追加: Hamming計算量（実際に計算したHamming距離の数）を追跡')

print('\n【PivotLB (Pipeline C) の特徴】')
print('- 閾値チューニング不要: 候補数Lを直接指定可能')
print('- 理論的保証: 三角不等式によりLB(q,x) <= d(q,x) が常に成立')
print('- 計算コスト: LB計算はHamming全探索の約1/16')
print('- 候補数制御: argpartitionにより正確にL件を選択')

print('\n【Pipeline H (Band+Probe+PivotLB) の位置づけ】')
print('- Band+Probeで候補をまず数千件に絞る（高速、粗い）')
print('- LBで候補をL件に精密フィルタ（Hamming計算なし）')
print('- L件のみHamming距離計算（精密、高コスト）')
print('- → 3段カスケード: Band+Probe → LB精密フィルタ → Hamming → Cosine')

print('\n【推奨パイプライン】')
for dataset_name in ['EN', 'JA', 'MiniLM']:
    results = all_results[f'{dataset_name}_L500']
    r_a = next(r for r in results if r['id'] == 'A')
    r_b = next(r for r in results if r['id'] == 'B')
    r_c = next(r for r in results if r['id'] == 'C')
    r_h = next(r for r in results if r['id'] == 'H')
    
    print(f'\n  [{dataset_name}]')
    print(f'    A (Baseline):    R@10={r_a["recall_at_k"]*100:.1f}%, HamCalc={r_a["hamming_computed"]:.0f}')
    print(f'    B (PvtRange):    R@10={r_b["recall_at_k"]*100:.1f}%, HamCalc={r_b["hamming_computed"]:.0f}')
    print(f'    C (PivotLB):     R@10={r_c["recall_at_k"]*100:.1f}%, HamCalc={r_c["hamming_computed"]:.0f}')
    print(f'    H (Band+PvtLB):  R@10={r_h["recall_at_k"]*100:.1f}%, HamCalc={r_h["hamming_computed"]:.0f}')

print('\n【実験84の結論の修正】')
print('- 「DF-LSHよりITQが良い」→ 「ITQ回転最適化がPCA符号化より量子化品質で優れる」（ハッシュ品質の話）')
print('- 「ITQ+Pivotが最良」→ 「フィルタ機構としてはLBベースのPivotが閾値不要で優れる」（フィルタの話）')
print('- Band+Probeの価値: 大規模データでのHamming計算量削減に有効')
print('- 統合パイプライン: Band+Probe → LB → Hamming → Cosine の4段カスケードが最も効率的')

print('\n【LB品質（Spearman相関）】')
for name, corr in lb_correlations.items():
    print(f'  {name}: LB-Hamming Spearman = {corr:.4f}')

Experiment 85: Pivot LB Fair Comparison - Summary

【実験84からの改善点】
1. 公平性: 全パイプラインがITQ 128bitハッシュを共有（ハッシュ品質差を排除）
2. 新手法: LB(q,x) = max_i |d(q,pi)-d(x,pi)| による三角不等式下界フィルタ
3. 評価軸追加: Hamming計算量（実際に計算したHamming距離の数）を追跡

【PivotLB (Pipeline C) の特徴】
- 閾値チューニング不要: 候補数Lを直接指定可能
- 理論的保証: 三角不等式によりLB(q,x) <= d(q,x) が常に成立
- 計算コスト: LB計算はHamming全探索の約1/16
- 候補数制御: argpartitionにより正確にL件を選択

【Pipeline H (Band+Probe+PivotLB) の位置づけ】
- Band+Probeで候補をまず数千件に絞る（高速、粗い）
- LBで候補をL件に精密フィルタ（Hamming計算なし）
- L件のみHamming距離計算（精密、高コスト）
- → 3段カスケード: Band+Probe → LB精密フィルタ → Hamming → Cosine

【推奨パイプライン】

  [EN]
    A (Baseline):    R@10=84.0%, HamCalc=9999
    B (PvtRange):    R@10=84.2%, HamCalc=9055
    C (PivotLB):     R@10=23.0%, HamCalc=500
    H (Band+PvtLB):  R@10=33.3%, HamCalc=500

  [JA]
    A (Baseline):    R@10=98.1%, HamCalc=9999
    B (PvtRange):    R@10=96.7%, HamCalc=6956
    C (PivotLB):     R@10=40.8%, HamCalc=500
    H (Band+PvtLB):  R@10=67.0%, HamCalc=500

  [MiniLM]
    A (Baseline):    R@10=98.3%, HamCalc=

## 11. 評価

### LBアプローチの評価: 期待vs現実

LB(q,x) = max_i |d(q,pi) - d(x,pi)| による三角不等式下界フィルタは、**理論的には正しいが、実用上は8ピボットでは不十分**であることが判明した。

#### LBの品質問題

| Dataset | LB-Hamming Spearman | LB Tightness (mean) | LB ≥ 50% of Hamming |
|---|---|---|---|
| EN | 0.2613 | 0.278 | 1.3% |
| JA | 0.3070 | 0.285 | 3.6% |
| MiniLM | 0.2119 | 0.261 | 1.8% |

- **Spearman相関が0.2-0.3と非常に弱い**: LBの順序がHamming距離の順序をほとんど保存していない
- **Tightness平均0.26-0.29**: LBは真のHamming距離の約27%しかカバーしない → 下界が緩すぎる
- **LB ≥ 50%の割合が1-4%**: ほとんどの場合、LBは真のHamming距離の半分以下

この結果、**LBによるtop-L選択は、真の近傍を大量に取りこぼす**。L=500でENのFilter Recallはわずか23.0%（Pipeline C）。

#### なぜLBが緩いのか

128bitハッシュ空間で8ピボットしか使わない場合、LBは128次元中の「最も情報量の多い1次元」しか反映しない。
三角不等式 `d(q,x) ≥ |d(q,pi) - d(x,pi)|` は各ピボットの1次元射影に過ぎず、128次元の距離情報のごく一部。

**必要ピボット数の推定**: Spearmanを0.7以上にするには、数十〜数百ピボットが必要と推測される（ただしピボット距離の保存コストが増大）。

#### Range方式が機能する理由

一方で、PivotRange(t=20)はFilter Recall 96.7-99.2%を維持している。これはRangeが「近い文書を見つける」のではなく「明らかに遠い文書を除外する」フィルタとして機能しているため。
- LB: 「最もLBが小さいL件を選ぶ」→ LBが緩いためランキングが不正確
- Range: 「全ピボットで±20以内の文書を残す」→ 明らかに遠い文書をAND条件で除外 → 安全な篩い落とし

#### Pipeline H（Band+Probe+PivotLB）の位置づけ

Band+Probe(16)で候補を1000-3700件に絞った後にLBを適用するPipeline Hは、Pipeline Cよりは良い結果を示す:
- JA L=1000: H=81.8% vs G=82.8%（ほぼ同等）
- JA L=500: H=67.0% vs G=82.6%（大きな差）

Band+Probeが既に候補を絞った後では、LBの相対的な品質が向上するが、L=500以下では依然としてRange方式に劣る。

### 実験84からの結論修正

#### 確認された知見（変更なし）
1. **ITQ回転最適化がPCA符号化より量子化品質で優れる**（ハッシュ品質の話）
2. **Band+Probeによる候補絞り込みは有効**（62-87%削減でRecall維持）
3. **PivotRange(t=20-25)はRecallを殆ど損なわずに候補を削減**

#### 新たな知見
1. **LBベースのPivotフィルタは8ピボットでは実用的でない**: Spearman 0.2-0.3の相関では、top-L選択が機能しない
2. **Range方式は「除外フィルタ」として合理的**: LBが緩くても、AND条件による除外は安全に機能
3. **Pipeline H（Band+Probe+PvtLB）はPareto上に乗る**: 95%削減で33-67%のR@10を達成（極端な効率要求向け）
4. **最良の組合せはBand+Probe(16)**: 62-87%削減、71-83%のR@10（Rangeの追加効果は小さい）

### 推奨パイプライン（修正版）

| 優先事項 | 推奨パイプライン | R@10範囲 | Hamming計算量 |
|---|---|---|---|
| 精度最優先 | A (Hamming全探索) or B (PvtRange) | 84-98% | N全件 |
| バランス | F (Band+Probe(16)) | 71-83% | ~1250-3757件 |
| 効率最優先 | H (Band+Probe+PvtLB) | 33-67% | 500件固定 |

### 今後の改善方向

1. **ピボット数の増加**: 8→32→128ピボットでLBのSpearman改善を検証
2. **ピボット距離の効率的な格納**: uint8 × 32ピボット = 256byte/doc（許容範囲）
3. **LBの適応的閾値**: LBをRangeのように「除外フィルタ」として使う（`LB(q,x) > adaptive_threshold → exclude`）
4. **大規模データでの検証**: 10Kデータでは全探索が十分高速のため、100K-1M規模での差別化が重要